In [9]:
import os
import glob

In [8]:
os.path.join('E:/s', 'abd', 'abc', 'd')

'E:/s\\abd\\abc\\d'

In [25]:
os.listdir("E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\")

['Ses01F_impro01_F000.wav',
 'Ses01F_impro01_F001.wav',
 'Ses01F_impro01_F002.wav',
 'Ses01F_impro01_F003.wav',
 'Ses01F_impro01_F004.wav',
 'Ses01F_impro01_F005.wav',
 'Ses01F_impro01_F006.wav',
 'Ses01F_impro01_F007.wav',
 'Ses01F_impro01_F008.wav',
 'Ses01F_impro01_F009.wav',
 'Ses01F_impro01_F010.wav',
 'Ses01F_impro01_F011.wav',
 'Ses01F_impro01_F012.wav',
 'Ses01F_impro01_F013.wav',
 'Ses01F_impro01_F014.wav',
 'Ses01F_impro01_F015.wav',
 'Ses01F_impro01_M000.wav',
 'Ses01F_impro01_M001.wav',
 'Ses01F_impro01_M002.wav',
 'Ses01F_impro01_M003.wav',
 'Ses01F_impro01_M004.wav',
 'Ses01F_impro01_M005.wav',
 'Ses01F_impro01_M006.wav',
 'Ses01F_impro01_M007.wav',
 'Ses01F_impro01_M008.wav',
 'Ses01F_impro01_M009.wav',
 'Ses01F_impro01_M010.wav',
 'Ses01F_impro01_M011.wav',
 'Ses01F_impro01_M012.wav',
 'Ses01F_impro01_M013.wav']

In [18]:
"asbdiasf.wav"[:-4]

'asbdiasf'

In [19]:
a = []
b = [1, 2, 3]

In [21]:
a + b + b

[1, 2, 3, 1, 2, 3]

In [1]:
from data.mfcc_extraction import IEMOCAP_Dataset

ROOT = "E:\\Download\\IEMOCAP_full_release_withoutVideos\\IEMOCAP_full_release"
iemocap = IEMOCAP_Dataset(root=ROOT)

In [3]:
len(iemocap.utterance_list)

10039

In [3]:
len(iemocap.label_map.items())

10039